In [2]:
import pandas as pd

In [25]:
adv_stats = pd.read_csv("adv_stats.csv")
hist_odds = pd.read_csv("hist_odds.csv")
mom_stats = pd.read_csv("momentum.csv")
today_lines = pd.read_csv("today_lines.csv")
team_id = {'Washington' : 1610612764, 'GoldenState' : 1610612744, 'Minnesota' : 1610612750, 'Houston' : 1610612745, 'Milwaukee' : 1610612749, 'Chicago' : 1610612741, 'Philadelphia': 1610612755, 'Memphis' : 1610612763, 'Brooklyn' : 1610612751, 
           'OklahomaCity' : 1610612760, 'Sacramento' : 1610612758, 'Charlotte' : 1610612766, 'Indiana' : 1610612754, 'SanAntonio' : 1610612759, 'Orlando' : 1610612753, 'Utah' : 1610612762, 'NewOrleans' : 1610612740, 'LALakers': 1610612747, 'Portland' : 1610612757, 
           'Dallas' : 1610612742, 'Atlanta' : 1610612737, 'Boston' : 1610612738, 'Cleveland' : 1610612739, 'Detroit': 1610612765, 'Miami' : 1610612748, 'LAClippers' : 1610612746, 'Denver' : 1610612743, 'Phoenix' : 1610612756, 'NewYork' : 1610612752}
hist_odds = hist_odds.replace(team_id)
hist_odds['win'] = [1 if y > x else 0 for x,y in zip(hist_odds['Final_V'], hist_odds['Final_H'])]
home_team = hist_odds[["VH_y", "Team_H", "Final_H"]]
away_team = hist_odds[['VH_x', "Team_V", "Final_V"]]

home_team = pd.merge(left = home_team, right = adv_stats, left_on = "Team_H", right_on = "TEAM_ID", how = "left")
away_team = pd.merge(left = away_team, right = adv_stats, left_on = "Team_V", right_on = "TEAM_ID", how = "left")
model_stats = pd.merge(left = home_team, right = away_team, left_index = True, right_index = True, how = "outer")


     VH_y      Team_H  Final_H  Unnamed: 0_x             TEAM_NAME_x  \
0       1  1610612751      125           8.0           Brooklyn Nets   
1       1  1610612747      109          18.0      Los Angeles Lakers   
2       1  1610612739      121          23.0     Cleveland Cavaliers   
3       1  1610612754      121          12.0          Indiana Pacers   
4       1  1610612753      113          15.0           Orlando Magic   
..    ...         ...      ...           ...                     ...   
528     1  1610612759      102          14.0       San Antonio Spurs   
529     1  1610612740       93          17.0    New Orleans Pelicans   
530     1  1610612757      123          19.0  Portland Trail Blazers   
531     1  1610612752      114          29.0         New York Knicks   
532     1  1610612756      120          28.0            Phoenix Suns   

        TEAM_ID_x  GP_x   W_x   L_x  W_PCT_x  ...  MIN_RANK_y  \
0    1.610613e+09  37.0  24.0  13.0    0.649  ...         4.0   
1    